In [1]:
import os
%pwd

'/home/user/Desktop/Main Projects/Disease-classification-Using-Deep-Learning-MLOps/research'

In [2]:
os.chdir("../")
%pwd

'/home/user/Desktop/Main Projects/Disease-classification-Using-Deep-Learning-MLOps'

In [3]:
#Entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from classifier.constants import *
from classifier.utils.common import read_yaml, create_directories

In [5]:
#config manager in src
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
config_ = ConfigurationManager()
ing = config_.get_data_ingestion_config()
ing

[2023-11-07 10:34:32,786: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-07 10:34:32,792: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-07 10:34:32,798: INFO: common: created directory at: artifacts]
[2023-11-07 10:34:32,801: INFO: common: created directory at: artifacts/data_ingestion]


DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://drive.google.com/file/d/1CH3yvI6ycIUjna-csPZE9X56yexzh-_f/view?usp=drive_link', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')

In [8]:
ing.root_dir

'artifacts/data_ingestion'

In [7]:
import os
import zipfile
import gdown
from classifier import logger
from classifier.utils.common import get_size

In [8]:
#Component
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
# pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-06 17:52:08,623: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-06 17:52:08,740: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-06 17:52:08,770: INFO: common: created directory at: artifacts]
[2023-11-06 17:52:08,772: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-06 17:52:08,773: INFO: 3763391873: Downloading data from https://drive.google.com/file/d/1CH3yvI6ycIUjna-csPZE9X56yexzh-_f/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1CH3yvI6ycIUjna-csPZE9X56yexzh-_f
From (redirected): https://drive.google.com/uc?/export=download&id=1CH3yvI6ycIUjna-csPZE9X56yexzh-_f&confirm=t&uuid=0ba66b65-ad33-450d-ac34-ce71287027de
To: /home/user/Desktop/Main Projects/Disease-classification-Using-Deep-Learning-MLOps/artifacts/data_ingestion/data.zip
100%|██████████| 61.8M/61.8M [00:12<00:00, 4.85MB/s]


[2023-11-06 17:52:23,562: INFO: 3763391873: Downloaded data from https://drive.google.com/file/d/1CH3yvI6ycIUjna-csPZE9X56yexzh-_f/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
